In [1]:
import pandas as pd
import numpy as np
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
request = drive_service.files().get_media(fileId = '1XQgPGGrzOU24SGcujJOZ9mPpDDfjaV4o').execute()
import io
data = pd.read_csv(io.BytesIO(request))
data.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [2]:
data[data['Cancelled'] == 1].head(20)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
178,2008,1,3,4,NaN,700,NaN,830,WN,126,NaN,NaN,90.0,NaN,NaN,NaN,LAS,OAK,407,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
373,2008,1,3,4,NaN,1100,NaN,1215,WN,1146,NaN,NaN,75.0,NaN,NaN,NaN,LAX,OAK,337,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
399,2008,1,3,4,NaN,905,NaN,1025,WN,469,NaN,NaN,80.0,NaN,NaN,NaN,LAX,SFO,337,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
401,2008,1,3,4,NaN,1620,NaN,1740,WN,618,NaN,NaN,80.0,NaN,NaN,NaN,LAX,SFO,337,NaN,NaN,1,C,0,NaN,NaN,NaN,NaN,NaN
415,2008,1,3,4,NaN,1930,NaN,2035,WN,2528,NaN,NaN,65.0,NaN,NaN,NaN,LAX,SJC,308,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
676,2008,1,3,4,NaN,845,NaN,1140,WN,437,NaN,NaN,115.0,NaN,NaN,NaN,MDW,BDL,777,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
686,2008,1,3,4,NaN,1925,NaN,2045,WN,934,NaN,NaN,80.0,NaN,NaN,NaN,MDW,BNA,395,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
727,2008,1,3,4,NaN,600,NaN,805,WN,3326,NaN,NaN,65.0,NaN,NaN,NaN,MDW,DTW,229,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
881,2008,1,3,4,NaN,1110,NaN,1215,WN,1402,NaN,NaN,65.0,NaN,NaN,NaN,MDW,STL,251,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN
884,2008,1,3,4,NaN,1445,NaN,1550,WN,2205,NaN,NaN,65.0,NaN,NaN,NaN,MDW,STL,251,NaN,NaN,1,A,0,NaN,NaN,NaN,NaN,NaN


In [0]:
data = data.drop(columns=['CancellationCode', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Year'])
data = data.dropna(subset=['DepTime', 'ArrTime', 'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
                          'DepDelay', 'TaxiIn', 'TaxiOut'])
# Tạo cột mới với encode cho target column ArrDelay
data['ArrDelay_ecd'] = np.where(data['ArrDelay'] >= 30, 1, 0)
# xóa cột ArrDelay đi vì ta đã có cột ArrDelay_ecd
data = data.drop(columns=['ArrDelay'])

In [4]:
# check missing value again
missing_val_in_column = (data.isnull().sum())
print(missing_val_in_column[missing_val_in_column >0])

Series([], dtype: int64)


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [0]:
data = data.drop(columns=['TailNum', 'Origin', 'Dest','DepDelay', 'Cancelled','Diverted'])

In [7]:
data.head()

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,Distance,TaxiIn,TaxiOut,ArrDelay_ecd
0,1,3,4,2003.0,1955,2211.0,2225,WN,335,128.0,150.0,116.0,810,4.0,8.0,0
1,1,3,4,754.0,735,1002.0,1000,WN,3231,128.0,145.0,113.0,810,5.0,10.0,0
2,1,3,4,628.0,620,804.0,750,WN,448,96.0,90.0,76.0,515,3.0,17.0,0
3,1,3,4,926.0,930,1054.0,1100,WN,1746,88.0,90.0,78.0,515,3.0,7.0,0
4,1,3,4,1829.0,1755,1959.0,1925,WN,3920,90.0,90.0,77.0,515,3.0,10.0,1


In [8]:
data['ArrDelay_ecd'].value_counts()

0    5927611
1     927413
Name: ArrDelay_ecd, dtype: int64

In [9]:
len(data.columns)

16

In [10]:
from sklearn.utils import resample
data_down = resample(data[data['ArrDelay_ecd'] == 0], n_samples= 2074663, random_state=0) # down orginal into 35%
data_down = data_down.append(resample(data[data['ArrDelay_ecd'] == 1], n_samples= 324594, random_state=0)) # down orginal into 35%)
data_down['ArrDelay_ecd'].value_counts()                   

0    2074663
1     324594
Name: ArrDelay_ecd, dtype: int64

Giảm giữ liệu xuống còn 35% so với dữ liệu gốc

In [0]:
one_hot = pd.get_dummies(data= data_down , columns=['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier'])

In [12]:
len(one_hot.columns)

82

In [0]:
data_down = one_hot
del one_hot, data

In [0]:
from numpy import count_nonzero
A = np.array(data_down)
# calculate sparsity
sparsity = 1.0 - np.count_nonzero(A) / A.size
print(sparsity)

0.8154241846233352


### Apply SVD

In [0]:
from sklearn.decomposition import TruncatedSVD

In [0]:
X = data_down.drop(['ArrDelay_ecd'], axis=1)
y = data_down["ArrDelay_ecd"]
svd = TruncatedSVD(n_components=30)
svd.fit(X)
new_X = svd.transform(X)

**Hold out**

In [0]:
x_train, x_val, y_train, y_val = train_test_split(new_X, y, test_size = 0.2, random_state = 0)

In [0]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.96      0.93    414765
           1       0.62      0.37      0.46     65087

    accuracy                           0.88    479852
   macro avg       0.76      0.67      0.70    479852
weighted avg       0.87      0.88      0.87    479852



In [0]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.93      1.00      0.96    414765
           1       0.99      0.53      0.69     65087

    accuracy                           0.93    479852
   macro avg       0.96      0.76      0.82    479852
weighted avg       0.94      0.93      0.93    479852



In [0]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    414765
           1       0.94      0.95      0.94     65087

    accuracy                           0.98    479852
   macro avg       0.97      0.97      0.97    479852
weighted avg       0.98      0.98      0.98    479852



In [0]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.99      1.00      0.99    414765
           1       0.97      0.92      0.94     65087

    accuracy                           0.99    479852
   macro avg       0.98      0.96      0.97    479852
weighted avg       0.99      0.99      0.99    479852



In [0]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(x_train, y_train)
y_pred = gbk.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96   1185258
           1       0.96      0.52      0.68    185747

    accuracy                           0.93   1371005
   macro avg       0.95      0.76      0.82   1371005
weighted avg       0.93      0.93      0.92   1371005



In [0]:
# Support Vector Machines
from sklearn.svm import SVC

svc = SVC()
svc.fit(x_train, y_train)
y_pred = svc.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


### Apply Select K best

In [0]:
from sklearn.feature_selection import SelectKBest, f_classif

In [0]:
X = data_down.drop(['ArrDelay_ecd'], axis=1)
y = data_down["ArrDelay_ecd"]
new_X = SelectKBest(f_classif, k=20).fit_transform(X, y)

**Hold out**

In [0]:
x_train, x_val, y_train, y_val = train_test_split(new_X, y, test_size = 0.2, random_state = 0)

In [21]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.84      0.87    414765
           1       0.31      0.46      0.37     65087

    accuracy                           0.79    479852
   macro avg       0.61      0.65      0.62    479852
weighted avg       0.83      0.79      0.80    479852



In [20]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.93      1.00      0.96    414765
           1       0.98      0.52      0.68     65087

    accuracy                           0.93    479852
   macro avg       0.96      0.76      0.82    479852
weighted avg       0.94      0.93      0.93    479852



In [21]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    414765
           1       0.97      0.97      0.97     65087

    accuracy                           0.99    479852
   macro avg       0.98      0.98      0.98    479852
weighted avg       0.99      0.99      0.99    479852



In [22]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.98      1.00      0.99    414765
           1       0.98      0.89      0.94     65087

    accuracy                           0.98    479852
   macro avg       0.98      0.95      0.96    479852
weighted avg       0.98      0.98      0.98    479852



In [23]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(x_train, y_train)
y_pred = gbk.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96    414765
           1       0.96      0.51      0.67     65087

    accuracy                           0.93    479852
   macro avg       0.94      0.76      0.82    479852
weighted avg       0.93      0.93      0.92    479852



In [23]:
# Support Vector Machines
from sklearn.svm import LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93    414765
           1       0.99      0.07      0.13     65087

    accuracy                           0.87    479852
   macro avg       0.93      0.53      0.53    479852
weighted avg       0.89      0.87      0.82    479852



# Conclusions of this section

- Khi down sample của dữ liệu xuống còn **35%** so với dữ liệu gốc (theo tỷ lệ của giá trị 1,0 trong target column ArrDelay_ecd) thì tốc độ tính toán và chạy mô hình nhanh hơn hẳn và kết quả dự đoán của các mô hình không thay đổi nhiều, chênh lệch tầm 1-2%.

- Khi dùng SVD cho bộ dữ liệu (n_components=30) thì kết quả so với dùng nguyên dữ liệu (raw data) bị _giảm xuống một chút_. Ở đây ta dùng SVD thay vì dùng PCA là bởi sau khi hot encode các columns category ('Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier') thì bộ dữ liệu lúc này đã biến thành **sparse matrix** (ma trận rời rạc) có **81.54%** là giá trị **0**.

- Nhưng khi dùng SelectKbest (k=20) thì mô hình train còn nhanh hơn nữa. Kết quả dự đoán của các mô hình so với phần test cũng giảm không đáng kể (khoảng 1%).

- Trong mô hình dùng Naive Bayes không thể chạy được multinomial Naive Bayes classifier bởi dữ liệu có **số âm **.

- Mô hình dùng Decision Tree phân loại tốt nhất.

Các câu hỏi được đặt ra như sau:

- Việc down sample của dữ liệu có được sử dụng trong thực tế vì dữ liệu lớn đòi hỏi tài nguyên máy tính lớn?

- Liệu bỏ hết cateogry column rồi dùng PCA thì mô hình liệu có tốt hơn?

-  SelectKbest (k=20) đã thể hiện tốt và tốc độ cũng như đưa ra dự đoán. Vậy k bao nhiêu phù hợp? Liệu PCA/SVD với cùng n_component = k của SelectKbest thì cái nào tốt hơn?

- Việc dữ liệu báo có giá trị âm sau khi đã xử lý là không hợp lý lắm. Liệu dữ liệu có sai sót chỗ nào đó mà ta chưa tìm hiểu ra?

-  Decision Tree phân loại tốt nhất ở bài toán này phân loại nhị phân, nếu nhãn phân loại là 3 hoặc 4 giá trị thì sao?  Decision Tree có còn tốt.